In [6]:
# BTC Gamma Scalping 回测可视化
import pandas as pd
import matplotlib.pyplot as plt
from data_loader import DataLoader
from engine import GammaScalping

# 参数设置
params = {
    "start_date": "2020-01-01",
    "end_date": "2023-12-31",
    "initial_capital": 1.0,
    "hedge_freq_days": 2
}

# 数据加载
loader = DataLoader("your_data.csv")
expiry = loader.get_third_friday(2020, 1)
df = loader.add_expiry_days(expiry)

# 策略初始化
engine = GammaScalping(df, params["initial_capital"], params["hedge_freq_days"])

results = []
for idx, row in df.iterrows():
    today = row['Date'].date()
    # 到期日建仓
    if today == row['Expiry']:
        if engine.current_position:
            engine.close_position(row)
        engine.open_position(row)
    # 对冲
    if engine.current_position and (engine.last_hedge_day is None or (today - engine.last_hedge_day).days >= params["hedge_freq_days"]):
        call_delta, put_delta, perp_delta, total_delta = engine.delta_hedging(row, today)
    else:
        call_delta = put_delta = perp_delta = total_delta = 0.0
    # 组合跟踪
    if engine.current_position:
        record = engine.track_portfolio(row, call_delta, put_delta, perp_delta, total_delta)
        results.append(record)

# 输出结果表
result_df = pd.DataFrame(results)
display(result_df)

# 净值曲线
plt.figure(figsize=(12, 6))
plt.plot(result_df['Date'], result_df['TotalAsset'])
plt.title("BTC Gamma Scalping 策略净值曲线")
plt.xlabel("日期")
plt.ylabel("总资产 (BTC)")
plt.grid()
plt.show()

# 最大回撤
roll_max = result_df['TotalAsset'].cummax()
drawdown = (result_df['TotalAsset'] - roll_max) / roll_max
max_drawdown = drawdown.min()
print(f"最大回撤: {max_drawdown:.2%}")

# 夏普比率
returns = result_df['TotalAsset'].pct_change().dropna()
sharpe = returns.mean() / returns.std() * (252 ** 0.5)
print(f"夏普比率: {sharpe:.2f}")

ModuleNotFoundError: No module named 'pandas'